In [ ]:
%matplotlib inline

# Add lat/lon coordinates to car parking SVGs, and order all parking SVGs by y-coodinate 

This notebook reads the car and bike parking SVGs for an array of cities, and reorders all polygons by y-coordinate. For cars, it additionally inserts the lat/lon coordinates of all polygons from mongodb.

Created on:  2017-04-13  
Last update: 2017-04-13  
Contact: michael.szell@moovel.com, michael.szell@gmail.com (Michael Szell)

## Preliminaries

### Parameters

In [25]:
citynames = ["amsterdam","berlin","boston","losangeles","newyork","portland","sanfrancisco","stuttgart","vienna"]
citynames = ["beijing", "budapest", "chicago", "copenhagen", "helsinki", "hongkong", "jakarta", "johannesburg", "london", "moscow", "rome", "singapore", "tokyo"]


### Imports

In [2]:
from __future__ import unicode_literals
import re
from operator import itemgetter
import sys
import csv
import os
import math
from random import shuffle, choice, uniform
import random
import pprint
pp = pprint.PrettyPrinter(indent=4)
from collections import defaultdict
import time
import datetime
import numpy as np
from numpy import *
from scipy import stats
import pyprind
import itertools
import logging
from collections import OrderedDict

import json
from xml.dom import minidom
from shapely.geometry import mapping, shape, LineString, LinearRing, Polygon, MultiPolygon
import shapely
import shapely.ops as ops
from shapely import affinity
from functools import partial
import pyproj
Projection = pyproj.Proj("+proj=merc +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs")
from scipy.ndimage.interpolation import rotate
from scipy.spatial import ConvexHull

import pymongo
from pymongo import MongoClient


In [26]:
for cityname in citynames:
    for mode in ["bike", "car"]:
        pathdataout = 'output/'+cityname+'/'+mode+'out/'
        
        # RE-ORDER BY Y-VALUE
        polygonstring = {}
        yxvals = []
        pids = []
        with open(pathdataout + "all.svg") as f:
            content = f.readlines()
        content = [x.strip() for x in content] 
        header = content[0]
        del content[0]
        for line in content[:-1]:
            pid = int(re.findall ( 'id="(.*?)"', line, re.DOTALL)[0])
            pids.append(pid)
            polygonstring[pid] = line
            points = re.findall ( 'points="(.*?)" moovel_rot=', line, re.DOTALL)[0].strip().split(" ")
            points = [(x.split(",")) for x in points]
            points = [(float(x[0]),float(x[1])) for x in points]
            yxvals.append((min(points,key=itemgetter(1))[1], min(points,key=itemgetter(0))[0]))

        # order lexicographically
        sortorder = sorted(range(len(yxvals)),key=lambda x:yxvals[x])
        svg = header+"\n"
        for s in sortorder:
            svg += polygonstring[pids[s]]+"\n"
        svg += "</svg>"

        # export (overwrite!)
        with open(pathdataout + "all.svg", "w") as f:
            f.write(svg)

for cityname in citynames:
    for mode in ["car"]: # does not work for bike, is fixed in 06
        pathdataout = 'output/'+cityname+'/'+mode+'out/'
        # ADD LAT/LON COORDINATES OF POLYGONS
        client = MongoClient()
        db = client[cityname+'_derived']
        ways = db['ways']
        cursor = ways.find({"$and": [{"properties.amenity.amenity": "parking"}, {"geometry.type": "Polygon"}, {"properties_derived.area": { "$gte": 12 }}]}).sort("properties_derived.area",-1)
        numparkingareas = cursor.count()
        print("There are " + str(numparkingareas) + " " + mode + " parking spaces in " + cityname)
        
        with open(pathdataout + "all.svg") as f:
            content = f.readlines()
        content = [x.strip() for x in content] 
        header = content[0]
        del content[0]
        svg = header+"\n"
        for line in content[:-1]:
            pid = int(re.findall ( 'id="(.*?)"', line, re.DOTALL)[0])
            way = ways.find_one({"_id":pid})
            coo = np.array(way["geometry"]["coordinates"])
            svgstring = " pointslatlon=\""
            strxylist = [str(coo[i][0])+","+str(coo[i][1]) for i in range(coo.shape[0])]
            for s in strxylist:
                svgstring += s+" "
            svg += line[:-2] + svgstring + "\"" + line[-2:]
        svg += "</svg>"
        # export (overwrite!)
        with open(pathdataout + "all.svg", "w") as f:
            f.write(svg)

There are 478 car parking spaces in beijing
There are 1582 car parking spaces in budapest
There are 4683 car parking spaces in chicago
There are 1342 car parking spaces in copenhagen
There are 3398 car parking spaces in helsinki
There are 809 car parking spaces in hongkong
There are 255 car parking spaces in jakarta
There are 683 car parking spaces in johannesburg
There are 6023 car parking spaces in london
There are 5808 car parking spaces in moscow
There are 2529 car parking spaces in rome
There are 1430 car parking spaces in singapore
There are 6922 car parking spaces in tokyo
